In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
import tensorflow as tf
import keras
from keras import models
from keras import layers
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Conv2D , SeparableConv2D, MaxPooling2D , Flatten , Dropout , BatchNormalization, Activation
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array,array_to_img
from keras.callbacks import ReduceLROnPlateau 
from keras import backend as K
from keras import optimizers
from sklearn.metrics import classification_report, recall_score, precision_score, confusion_matrix, f1_score, accuracy_score
from tensorflow.keras.utils import plot_model
from IPython.display import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [7]:
train_data_dir = 'E:/oct_dataset/oct_dataset/data_final_all'

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Generate training data
train_gen = train_datagen.flow_from_directory(
    directory=train_data_dir,
    target_size=(224, 224),  # Adjust the target size according to your requirements
    batch_size=16,
    class_mode='categorical',
    subset='training'
)

# Generate validation data
val_gen = train_datagen.flow_from_directory(
    directory=train_data_dir,
    target_size=(224, 224),  # Adjust the target size according to your requirements
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)


Found 66788 images belonging to 4 classes.
Found 16696 images belonging to 4 classes.


In [9]:
image_size = (224, 224, 3)

In [10]:
base_model = tf.keras.applications.ResNet152V2(input_shape=image_size,
                                               include_top=False,
                                               weights="imagenet")

In [11]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.metrics import Metric
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
import tensorflow_probability as tfp

class F1Score(Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='true_positives', initializer='zeros')
        self.possible_positives = self.add_weight(name='possible_positives', initializer='zeros')
        self.predicted_positives = self.add_weight(name='predicted_positives', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        possible_positives = tf.reduce_sum(y_true)
        predicted_positives = tf.reduce_sum(y_pred)
        self.true_positives.assign_add(true_positives)
        self.possible_positives.assign_add(possible_positives)
        self.predicted_positives.assign_add(predicted_positives)

    def result(self):
        precision = self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.possible_positives + tf.keras.backend.epsilon())
        f1 = 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())
        return f1

In [12]:
class BayesianConv2D(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, activation=None, **kwargs):
        super(BayesianConv2D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.activation = tf.keras.activations.get(activation)
        self.conv2d = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, activation=activation, **kwargs)
        self.conv2d_flipout = tfp.layers.Convolution2DFlipout(filters=filters, kernel_size=kernel_size, activation=activation, **kwargs)

    def call(self, inputs):
        x = self.conv2d(inputs)
        return x

    def compute_output_shape(self, input_shape):
        return self.conv2d.compute_output_shape(input_shape)

    def get_config(self):
        config = super(BayesianConv2D, self).get_config()
        config.update({
            'filters': self.filters,
            'kernel_size': self.kernel_size,
            'activation': self.activation,
        })
        return config

In [13]:
class BayesianDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super(BayesianDense, self).__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        self.dense_flipout = tfp.layers.DenseFlipout(units, activation=activation, **kwargs)

    def call(self, inputs):
        x = self.dense_flipout(inputs)
        return x

    def compute_output_shape(self, input_shape):
        return self.dense_flipout.compute_output_shape(input_shape)

    def get_config(self):
        config = super(BayesianDense, self).get_config()
        config.update({
            'units': self.units,
            'activation': self.activation,
        })
        return config


In [14]:
from keras.layers import Input

model = Sequential([
    Input(shape=(224, 224, 3)),
    BayesianConv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    BayesianConv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    BayesianConv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    BayesianDense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    BayesianDense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(4, activation='softmax')  # Adjust output units based on the number of classes
])


C:\Users\PMLS\anaconda3\Lib\site-packages\tensorflow_probability\python\layers\util.py:98: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  loc = add_variable_fn(
C:\Users\PMLS\anaconda3\Lib\site-packages\tensorflow_probability\python\layers\util.py:108: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  untransformed_scale = add_variable_fn(


In [15]:
model.compile(loss=CategoricalCrossentropy(), optimizer=Adam(), metrics=['accuracy', F1Score()])

In [16]:
earlystop = EarlyStopping(verbose=1, patience=5)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.8, min_lr=1e-6)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', verbose=1, save_best_only=True, mode='min')


In [28]:
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
import tensorflow_addons as tfa
import tensorflow as tf

num_classes = 4

for epoch in range(10):
    lr = icda_strategy(epoch)
    model.compile(
        loss=CategoricalCrossentropy(),
        optimizer=Adam(learning_rate=lr),
        metrics=[Accuracy(), tfa.metrics.F1Score(num_classes=num_classes, average='macro')]
    )
    model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=1,
        callbacks=[
            EarlyStopping(patience=10, monitor='val_loss', restore_best_weights=True),
            ReduceLROnPlateau(factor=0.5, patience=5, monitor='val_loss'),
            ModelCheckpoint(filepath='model_checkpoint.keras', monitor='val_loss', save_best_only=True)
        ]
    )

# Ensure that the RandomNormal initializer is seeded.
initializer = RandomNormal(mean=0.0, stddev=0.05, seed=42)

# Update any custom layer or model class to use 'add_weight' instead of 'add_variable'.
# Example of defining a custom layer
class CustomLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.loc = self.add_weight(name='loc', i2nitializer=initializer, shape=(1,), dtype=tf.float32) 
        self.untransformed_scale = self.add_weight(name='untransformed_scale', initializer=initializer, shape=(1,), dtype=tf.float32)

# If you have a custom PyDataset or similar class, ensure it calls super().__init__.
class PyDataset:
    def __init__(self, **kwargs):
        super(PyDataset, self).__init__(**kwargs)
        # Your initialization code


4175/4175 [==============================] - ETA: 0s - loss: 1.2637 - accuracy: 0.0000e+00 - f1_score: 0.1573

MemoryError: 

In [ ]:
complete_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip = True,
        )

complete_generator = complete_datagen.flow_from_directory(
        directory = train_data_dir,
        target_size=(224, 224),
        batch_size=1,
        class_mode=None,
        )

In [ ]:
import numpy as np

# Calculate the number of steps needed to cover the entire dataset
STEP_SIZE_COMPLETE = complete_generator.n // complete_generator.batch_size

# Predict class probabilities for each sample in the dataset
train_probs = model.predict(complete_generator, steps=STEP_SIZE_COMPLETE)

# Convert predicted probabilities to class labels (0, 1, 2, or 3)
train_preds = np.argmax(train_probs, axis=1)

19903/83484 ━━━━━━━━━━━━━━━━━━━━ 6:54:37 391ms/step

In [23]:
print(set(train_preds))

{0, 3}


In [24]:
np.unique(complete_generator.classes, return_counts=True)

(array([0, 1, 2, 3]), array([37205, 11348,  8616, 26315], dtype=int64))

In [25]:
for i in range(4):
    value = train_preds.count(i)
    print(f'Class {i} -> {value}')

Class 0 -> 83479
Class 1 -> 0
Class 2 -> 0
Class 3 -> 5
